In [13]:
import numpy as np

from chugunov_indicator.chugunov_2009 import chugunov_2009
from chugunov_indicator.analytic.predict import _screening_intercept, _skip_chugunov_2009
from chugunov_indicator import ScreeningFactorData

import pynucastro as pyna
import yt

rng = np.random.default_rng(1)

To see how effective the fitting described in `3. analytic.ipynb` (and implemented in `chugunov_indicator.analytic.predict.py`) is, we can test it on a `ScreeningFactorData` object.

In [14]:
test = ScreeningFactorData(total := 3*10**5, rng)

In [15]:
%%timeit
predict_skip = _skip_chugunov_2009(test.y0, test.abar, test.z2bar, test.z1, test.z2)

34.6 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
F = chugunov_2009(test.T, test.D, test.abar, 1, test.z2bar, test.z1, test.z2, 1, 1)

268 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
actual_skip = F < 1.01

In [20]:
p = np.count_nonzero(actual_skip)
n = 3*10**5 - p
print(
    f"Can skip: {p}/{total} ({100 * p/total:.2f}%)",
    f"Can't skip: {n}/{total} ({100 * n/total:.2f}%)",
    sep="\n"
)

Can skip: 104961/300000 (34.99%)
Can't skip: 195039/300000 (65.01%)


We find that screening can be skipped around half of the time, depending on our choices and the selected screening pair.

In [6]:
TP = predict_skip & actual_skip
TN = ~predict_skip & ~actual_skip
FP = predict_skip & ~actual_skip
FN = ~predict_skip & actual_skip

tp = np.count_nonzero(TP)
tn = np.count_nonzero(TN)
fp = np.count_nonzero(FP)
fn = np.count_nonzero(FN)

A breakdown of the data is given below.

In [11]:
print(
    f"True Positives: {tp}/{total} ({100 * tp/total:.2f}%)",
    f"True Negatives: {tn}/{total} ({100 * tn/total:.2f}%)",
    f"False Positives: {fp}/{total} ({100 * fp/total:.2f}%)",
    f"False Negatives: {fn}/{total} ({100 * fn/total:.2f}%)",
    sep="\n"
)

True Positives: 98296/300000 (32.77%)
True Negatives: 195039/300000 (65.01%)
False Positives: 0/300000 (0.00%)
False Negatives: 6665/300000 (2.22%)


In [12]:
print(
    f"Accuracy: {100 * (tp + tn)/(p + n):.2f}%",
    f"Precision: {100 * tp/(tp + fp):.2f}%",
    f"Recall: {100 * tp/(tp + fn):.2f}%",
    sep="\n"
)

Accuracy: 97.78%
Precision: 100.00%
Recall: 93.65%


Even though our model's prediction of the intercept $C$ isn't the most precise that it could be, the predictions for whether screening can be skipped were correct the vast majority of the time, with false negatives making up a very small fraction of the total. False negatives are fine, as it just means the screening factor will end up being computed and found to be between $1$ and $1.01$ anyway. False positives are the real issue, as those mean that screening will be skipped even in a scenario where it could be relevant - but fortunately, there were no false positives.